# Sunglint Computation analog to 6SV

In [11]:
import numpy as np
import drjit as dr
import mitsuba as mi
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

mi.set_variant("llvm_ad_rgb")

In [19]:
def friedman_sverdrup(chlorinity):
    return 0.00017492711 * (0.03 + 1.805 * chlorinity)

def fresnel(nr, ni, coschi, sinchi):
    s = nr * nr - ni * ni - sinchi * sinchi
    a1 = np.abs(s)
    a2 = np.sqrt(s ** 2 + 4 * nr * nr * ni * ni)
    u = np.sqrt(0.5 * np.abs(a1 + a2))
    v = np.sqrt(0.5 * np.abs(a2 - a1))
    Rr2 = ((coschi - u) ** 2 + v * v) / ((coschi + u) ** 2 + v * v)
    b1 = (nr * nr - ni * ni) * coschi
    b2 = 2 * nr * ni * coschi
    Rl2 = ((b1 - u) ** 2 + (b2 + v) ** 2) / ((b1 + u) ** 2 + (b2 - v) ** 2)
    R = (Rr2 + Rl2) / 2
    return R

def cox_munk(phw, zx, zy, wspd):
    sigmaC = 0.003 + 0.00192 * wspd
    sigmaU = 0.00316 * wspd
    C21 = 0.01 - 0.0086 * wspd
    C03 = 0.04 - 0.033 * wspd
    C40 = 0.40
    C22 = 0.12
    C04 = 0.23
    xe = (np.cos(phw) * zx + np.sin(phw) * zy) / np.sqrt(sigmaC)
    xn = (-np.sin(phw) * zx + np.cos(phw) * zy) / np.sqrt(sigmaU)
    xe2 = xe * xe
    xn2 = xn * xn
    coef = 1.0 - C21 / 2.0 * (xe2 - 1.0) * xn - C03 / 6.0 * (xn2 - 3) * xn
    coef = coef + C40 / 24.0 * (xe2 * xe2 - 6.0 * xe2 + 3.0)
    coef = coef + C04 / 24.0 * (xn2 * xn2 - 6.0 * xn2 + 3.0)
    coef = coef + C22 / 4.0 * (xe2 - 1.0) * (xn2 - 1.0)
    prob = coef / (2.0 * np.pi * np.sqrt(sigmaU) * np.sqrt(sigmaC)) * np.exp(-(xe2 + xn2) / 2.0)
    return prob

def sunglint(to, ti, po, pi, pu, wspd, nr, ni, chlor):
    phi = pi - po
    phw = pi - pu
    cs = np.cos(ti)
    cv = np.cos(np.pi / 2 - to)
    ss = np.sin(ti)
    sv = np.sin(np.pi / 2 - to)
    zx = -sv * np.sin(phi) / (cs + cv)
    zy = (ss + sv * np.cos(phi)) / (cs + cv)

    print(f'phi: {phi}, phw: {phw}')
    print(np.sin(phi))
    print(f'zx: {zx}, zy: {zy}')

    # Tilt
    tan_tilt = np.sqrt(zx * zx + zy * zy)
    tilt = np.arctan(tan_tilt)

    # Cox Munk term
    specular_prob = cox_munk(phw, zx, zy, wspd)
    
    # Fresnel term
    cos2chi = cv * cs + sv * ss * np.cos(phi)
    if cos2chi > 1.0:
        cos2chi = 0.999999999
    elif cos2chi < -1.0:
        cos2chi = -0.999999999
    coschi = np.sqrt(0.5 * (1.0 + cos2chi))
    sinchi = np.sqrt(0.5 * (1.0 - cos2chi))
    if coschi > 1.0:
        coschi = 0.999999999
    elif coschi < -1.0:
        coschi = -0.999999999
    if sinchi > 1.0:
        sinchi = 0.999999999
    elif sinchi < -1.0:
        sinchi = -0.999999999

    nr_corr = nr + friedman_sverdrup(chlor)
    R = fresnel(nr_corr, ni, coschi, sinchi)
    
    # Reflectance of sun glint
    return specular_prob, R, tilt, cs, cv

In [20]:
# Some parameters for the test case
#   Sun angle
azim_s = 0.0

#   Wind speed and direction
wind_speed = 10.0
azim_w = 0.0

#   Water properties (IOR and chlorinity)
n_real = 1.333
n_imag = 0.0
chlorinity = 19

#   Variable sensor position
res = 25
epsilon = 1.0 * np.pi / 180
azim_vs = np.linspace(0, 2 * np.pi, 2 * res)
zen_vs = np.linspace(0, np.pi / 2, res)
    

In [21]:
# Unit Test
spec, R, tilt, cs, cv = sunglint(0, 0, np.pi, 0, 0, 10, n_real, n_imag, chlorinity)
print(f"Specular probability: {spec}")
print(f"Reflectance: {R}")
print(f"Tilt: {tilt}")
print(f"Sun cosine: {cs}")
print(f"View cosine: {cv}")

phi: -3.141592653589793, phw: 0
-1.2246467991473532e-16
zx: 1.2246467991473532e-16, zy: -1.0
Specular probability: 2.923023658207816e-07
Reflectance: 0.028656196615431864
Tilt: 0.7853981633974483
Sun cosine: 1.0
View cosine: 6.123233995736766e-17


In [ ]:
# Computation
results = {}
solar_zeniths = np.linspace(0, np.pi / 4, 46)

idx = 0
max_idx = len(solar_zeniths)
visuals = ["specular", "fresnel", "tilt", "cosine_solar", "cosine_view", "reflectance"]

for zen_s in solar_zeniths:
    speculars = np.zeros((res, 2 * res))
    fresnels = np.zeros((res, 2 * res))
    tilts = np.zeros((res, 2 * res))
    cosine_s = np.zeros((res, 2 * res))
    cosine_v = np.zeros((res, 2 * res))
    sunglint_reflectances = np.zeros((res, 2 * res))
    for i, zen_v in enumerate(zen_vs):
        for j, azim_v in enumerate(azim_vs):
            #spec, R, tilt, cs, cv = sunglint(zen_v, zen_s, azim_v, azim_s, azim_w,
            #                wind_speed, n_real, n_imag, chlorinity)
            
            spec, R, tilt, cs, cv = sunglint(0, 0, azim_v, azim_s, azim_w,
                            wind_speed, n_real, n_imag, chlorinity)
            
            # Clamp the cosines to avoid numerical issues
            cs = np.clip(cs, 0, 1.0)
            cv = np.clip(cv, 0, 1.0)

            speculars[i, j] = spec
            fresnels[i, j] = R
            tilts[i, j] = tilt
            cosine_s[i, j] = cs
            cosine_v[i, j] = cv

            #reflectance = (np.pi * R * spec) / (4 * np.cos(tilt) ** 4)
            reflectance = (np.pi * R * spec) / (4.0 * cs * cv * np.cos(tilt) ** 4)
            sunglint_reflectances[i, j] = reflectance
    results[zen_s] = {
        visuals[0]: speculars,
        visuals[1]: fresnels,
        visuals[2]: tilts,
        visuals[3]: cosine_s,
        visuals[4]: cosine_v,
        visuals[5]: sunglint_reflectances
    }
    idx = idx + 1
    print(f"Progress: {idx}/{max_idx}")

In [ ]:
# Clear all files in cox_munk
import os

for file in os.listdir("glints"):
    os.remove(os.path.join("glints", file))

In [ ]:
titles = {
    "specular": "Cox-Munk Specular Probability",
    "fresnel": "Fresnel Coefficient",
    "tilt": "Tilt Angle",
    "cosine_solar": "Cosine of Solar Angle",
    "cosine_view": "Cosine of View Angle",
    "reflectance": "Sunglint Reflectance"
}

idx = 0
for solar_zenith, glint_data in results.items():
    for title, data in glint_data.items():
        ax = plt.gca()
        im = ax.imshow(data, cmap='turbo', interpolation='quadric', extent=[0, 2 * np.pi, 0, np.pi / 2])

        # Name the axes
        plt.xlabel("Outgoing Azimuth")
        plt.ylabel("Outgoing Zenith")

        # Add the title
        degrees = np.degrees(solar_zenith)

        # Round to 2 decimal places
        degrees = round(degrees, 2)

        # Create x_axis tick labels in degrees
        x_ticks = np.linspace(0, 2 * np.pi, 5)
        x_labels = np.degrees(x_ticks)
        x_labels = [round(label, 2) for label in x_labels]
        plt.xticks(x_ticks, x_labels)

        # Create y_axis tick labels in degrees
        y_ticks = np.linspace(0, np.pi / 2, 3)
        y_labels = np.degrees(y_ticks)
        y_labels = [round(label, 2) for label in y_labels]
        plt.yticks(y_ticks, y_labels)

        # Add the title
        plt.title(f'{titles[title]}  |  Incoming Zenith = {str(degrees)}°')

        # Add color bar with same height as the plot
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.25)
        plt.colorbar(im, cax=cax)

        # Save the plot
        filename = f'glints/{title}_{str(degrees)}.png'
        plt.savefig(filename)
        plt.close()
    idx = idx + 1
    print(f"Progress: {idx}/{max_idx}")

# Close the plot
plt.close()

In [ ]:
# Create gif animation
import imageio

for visual in visuals:
    print(f'Processing visual: {visual}')

    filesnames = [f'glints/{visual}_{str(round(np.degrees(zenith), 2))}.png' for zenith in solar_zeniths]

    images = [imageio.imread(filename) for filename in filesnames]
    imageio.mimsave(f'glints/{visual}.gif', images, duration=5)

In [ ]:
# display the animation
from IPython.display import Image

visual = visuals[4]
animation = Image(f'glints/{visual}.gif')
animation

In [ ]:
# View a certain frame of the animation
visual = visuals[2]
frame = 0
image = Image(f'glints/{visual}_{str(round(np.degrees(solar_zeniths[frame]), 2))}.png')
image

In [ ]:
results[0.0]["reflectance"].dtype